In [13]:
import os
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap, RunnableLambda, RunnableParallel
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser


import re
from datetime import datetime

## Runnable Passthrough

In [15]:
chain = RunnablePassthrough()  | RunnablePassthrough()

x = chain.invoke({"user_input": "langchain"})

print(x)    

{'user_input': 'langchain'}


### Runnable lambda

In [16]:
def augment_query(input):
    user_input = input["user_input"]
    user_segment = input.get("user_segment", "regular")

    current_date = datetime.today().strftime("%Y-%m-%d")
    final_prompt = f"""System Message: Today is {current_date}. The user is a {user_segment} customer.

    
    User Question: {user_input}"""

    return {"prompt": final_prompt}

In [18]:
chain = RunnableLambda(augment_query)


input_data = {
    "user_input": "Do you have any offers on iPhones today?",
    "user_segment": "premium"
}

# Step 4: Invoke
augmented_prompt = chain.invoke(input_data)

print(augmented_prompt)

{'prompt': 'System Message: Today is 2025-07-12. The user is a premium customer.\n\n    \n    User Question: Do you have any offers on iPhones today?'}


## Runnable parallel

In [5]:
def get_browsing_history(input):
    user_id = input.get("user_id", "")
    # Simulated history
    if user_id == "vip_001":
        history = ["smartwatch", "fitness band", "apple watch series 9"]
    else:
        history = ["bluetooth speaker", "headphones"]
    return {"recently_viewed": history}


# Simulated user profile fetch
def get_user_profile(input):
    user_id = input.get("user_id", "guest")
    return {
        "user_id": user_id,
        "is_premium": user_id.startswith("vip"),
        "location": "Mumbai"
    }

In [12]:
parallel_chain = RunnableParallel({
    "browsing_info": RunnableLambda(get_browsing_history),
    "user_info": RunnableLambda(get_user_profile)
})

# Input
input_data = {
    "user_input": "Can you suggest something good?",
    "user_id": "vip_001"
}

# Run
result = parallel_chain.invoke(input_data)
print(result)

{'browsing_info': {'recently_viewed': ['smartwatch', 'fitness band', 'apple watch series 9']}, 'user_info': {'user_id': 'vip_001', 'is_premium': True, 'location': 'Mumbai'}}
